In [38]:
import tensorflow as tf

from keras import optimizers, layers, models

In [39]:
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(784, activation='sigmoid'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

In [40]:
def build_discriminator(img_shape):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [41]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [42]:
# Wymiary obrazów
img_shape = (28, 28, 1)

# Wymiar wektora szumu (wejście generatora)
latent_dim = 100

In [43]:
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(), metrics=['accuracy'])

In [44]:
generator = build_generator(latent_dim)
discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam())

In [45]:
# Przykładowe dane do trenowania (np. dane z zestawu MNIST)
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images / 127.5 - 1.0
train_images = train_images.reshape((60000, 28, 28, 1))

In [50]:
# Proces trenowania GAN
def train_gan(generator, discriminator, gan, epochs, batch_size):
    for epoch in range(epochs):
        # Generowanie losowego szumu dla generatora
        noise = tf.random.normal((batch_size, latent_dim))
        
        # Generowanie danych za pomocą generatora
        generated_images = generator.predict(noise)
        
        # Losowanie próbek z danych rzeczywistych
        idx = tf.random.uniform((batch_size,), 0, 60000, dtype=tf.int32)
        real_images = train_images[idx]
        
        # Tworzenie etykiet dla danych rzeczywistych i wygenerowanych
        real_labels = tf.ones((batch_size, 1))
        fake_labels = tf.zeros((batch_size, 1))
        
        # Trenowanie dyskryminatora na danych rzeczywistych
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        
        # Trenowanie dyskryminatora na danych wygenerowanych
        d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
        
        # Obliczanie całkowitej straty dyskryminatora
        d_loss = 0.5 * tf.add(d_loss_real, d_loss_fake) # type: ignore
        
        # Generowanie nowego szumu dla GAN
        noise = tf.random.normal((batch_size, latent_dim))
        
        # Tworzenie etykiet dla danych wygenerowanych (udawanych za prawdziwe)
        valid_labels = tf.ones((batch_size, 1))
        
        tf.keras.utils.disable_interactive_logging()
        # Trenowanie GAN poprzez aktualizację wag generatora przy zamrożonym dyskryminatorze
        g_loss = gan.train_on_batch(noise, valid_labels)
        
        # Wyświetlanie postępu co 100 epok
        if epoch % 500 == 0:
            print(f"Epoka {epoch}, Strata dyskryminatora: {d_loss[0]}, Strata generatora: {g_loss}")


In [51]:
# Trenowanie GAN
train_gan(generator, discriminator, gan, epochs=5000, batch_size=64)

Epoka 0, Strata dyskryminatora: 0.00019731737847905606, Strata generatora: 18.583696365356445
Epoka 500, Strata dyskryminatora: 1.076653916243231e-06, Strata generatora: 13.049737930297852
Epoka 1000, Strata dyskryminatora: 2.9081218144710874e-07, Strata generatora: 14.354414939880371
Epoka 1500, Strata dyskryminatora: 1.1305029090635799e-07, Strata generatora: 15.305027961730957
Epoka 2000, Strata dyskryminatora: 5.764339761071824e-08, Strata generatora: 15.979569435119629
Epoka 2500, Strata dyskryminatora: 3.036815598989051e-08, Strata generatora: 16.617355346679688
Epoka 3000, Strata dyskryminatora: 1.0002709771583795e-08, Strata generatora: 17.730072021484375
Epoka 3500, Strata dyskryminatora: 1.220822287706369e-08, Strata generatora: 17.529348373413086
Epoka 4000, Strata dyskryminatora: 6.570665789240593e-09, Strata generatora: 18.14812660217285
Epoka 4500, Strata dyskryminatora: 3.930192171708313e-09, Strata generatora: 18.662477493286133


: 